In [118]:
from juritagger import JuriMatcher
%load_ext autoreload
%autoreload 2
jm = JuriMatcher(["./dicos/merged.txt", "./dicos/cluster_merged_classif.json"], "fr_core_news_md")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:

fiche_text = """Fermeture de l'entreprise pour congés: comment le salarié est-il indemnisé?
L'indemnisation du salarié dépend du nombre de jours de fermeture
de l'entreprise et du nombre de jours de congés acquis par le salarié.
Si le salarié possède suffisamment de jours de congés par rapport au
nombre de jours de fermeture de congés, il perçoit l'indemnité de congés payés dans les conditions habituelles. 
Si le salarié n'a pas acquis suffisamment de jours de congés payés pour être indemnisé
durant l'intégralité de la fermeture de l'entreprise, Pôle emploi peut verser, sous conditions,
une aide financière pour congés non payés. Cette aide pour congés non payés est versée au salarié qui percevait,
avant sa reprise d'emploi, l'allocation d'aide au retour à l'emploi (ARE)  ou l'allocation de solidarité spécifique
(ASS). Si c'était le cas, le salarié doit formuler une demande d'aide auprès de l'agence Pôle emploi
dont il dépendait en tant que demandeur d'emploi. Pôle emploi pole_emploi Pôle emploi 
Le montant de l'aide est calculé par Pôle emploi qui tient compte du nombre de jours de fermeture
de l'entreprise; net des droits à congés payés acquis par le salarié au titre de son nouvel emploi.
 À défaut d'avoir droit à l'aide pour congés non payés, le salarié peut demander à son employeur de bénéficier 
de congés payés par anticipation, mais l'employeur n'est pas obligé d'accepter (sauf accord collectif ou
usages contraires). Le salarié peut également demander à bénéficier d'un congé sans solde ou d'une indemnité d'activité partielle,
en cas d'intempérie à caractère exceptionnel notamment."""

matches, doc = jm.tag_doc(fiche_text, mode = "class")

In [120]:
class MyDoc:
    def __init__(self, doc):
        self.doc = doc
        
    def __getitem__(self, item):
        return self.doc[item].text
    
    def _split(self, s=None):
        return self.doc.text.split(s)
    
    def __add__(self, o):
        return self.doc.text + o
    
    def __radd__(self, o):
        return o + self.doc.text

In [121]:
doc2 = MyDoc(doc)

In [122]:
def check_overlap(i, entity_range):
    v = []
    entity = entity_range[i]
    for j, a in enumerate(entity_range):
        if j != i and entity.intersection(a) != set():
            v.append(j)
    return v

def keep_longer_match(matches):
    entity_range = []
    for label, start, end in matches:
        entity_range.append(set(range(start, end)))

    new_matches = []
    for i in range(len(entity_range)):
        overlaps = check_overlap(i, entity_range)
        longer = [len(entity_range[over]) > len(entity_range[i]) for over in overlaps]
        equal = [len(entity_range[over]) == len(entity_range[i]) for over in overlaps]
        if any(longer):
            continue
            
        if any(equal):
            if matches[i] not in new_matches:
                new_matches.append(matches[i])
        
        else:
            new_matches.append(matches[i])
    return new_matches

In [123]:
matches = keep_longer_match(matches)

In [132]:
from display_entities import COLOR_LIST
options = {
    "ents": list(jm.entity_types),
    "colors" : {entity:COLOR_LIST[i] for i, entity in enumerate(jm.entity_types)}
}

In [133]:
from display_entities import serve_ents
serve_ents(doc2 , matches, options = options)


    Serving on port 5000...
    Using the 'ent' visualizer



127.0.0.1 - - [01/Mar/2019 11:37:11] "GET / HTTP/1.1" 200 18369
127.0.0.1 - - [01/Mar/2019 11:37:11] "GET /favicon.ico HTTP/1.1" 200 18369



    Shutting down server on port 5000.



In [126]:
list(jm.entity_types)

['CONTRAT',
 'TEMPS',
 'CHOMAGE',
 'AUTRE',
 'REMU',
 'RUPTURE',
 'ACCRO',
 'ENTREPRISE',
 'CONGE',
 'TRAVAIL',
 'RETRAITE',
 'EMPLOI',
 'MEDICAL',
 'PERS']